# 台灣外來入侵種資料集 工具箱

V0.2

## Init All 

In [1]:
from tiasdswak import *
from codes.lib import *

tiasdswak = TiasdSwak()

df_standard = tiasdswak.df_standard # 政府資料
df_append = tiasdswak.df_append # 民間資料庫
df_extend = tiasdswak.df_extend # 補充資料
df_mix = tiasdswak.df_mix # 政府+民間
df_merge = tiasdswak.df_merge # 政府+民間+補充
df_cur = tiasdswak.df_cur # 目前查詢結果
df_dig = tiasdswak.df_dig # 分析
df_coldef = tiasdswak.df_coldef #COL_DEF


TiasdSWAK-V0.2


In [ ]:
#df_dig['new']=1
#df_dig
#print("\n".join(list(df_merge.columns)))

#str1="','".join(tiasdswak.group['十大外來入侵物種'])
#str2="'%s'" %(str1)
#str2

#sql="select * from df"
#df1=sqldf(df_merge,sql)

#df_cur['name_code'].tolist()
#grp_info={"大花咸豐草":"9164c378-ef51-4423-a166-1a979c73a97d"}
#grp_info={"中國梨木蝨":"f7d026a6-719c-4c1b-ac04-f9e18b0a8abb"}
if 0:
    #df.to_csv("output/tbn_save.csv",index=False)
    df_merge.to_csv("output/台灣外來入侵種資料集 - 合併補充資料集.csv",index=False)
    
df_merge

## MIX 

In [ ]:
df_mix = df_standard
for row in df_append.iterrows():
    name_code = str(row[1]['name_code'])
    print(name_code)
    name_code_values=df_mix[df_mix['name_code']==name_code]['name_code'].values.tolist()
    if len(name_code_values) > 0: #delete match row
        print("%s exist!" %(name_code))
        idx=df_mix.index[df_mix['name_code']==name_code].tolist()
        df_mix.drop(idx, axis=0, inplace=True) 
df_mix=df_mix[df_mix['is_invasive']==1].append(df_append[df_append['is_invasive']==1])


## MERGE 

In [ ]:
df_extend = df_extend.astype({'name_code':'str'})
df_merge = df_mix.join(df_extend.set_index('name_code'),on='name_code',how='outer',lsuffix='L',rsuffix='R')
#df_merge.to_csv("/Users/wuulong/prg/tiasdswak/include/台灣外來入侵種資料庫補充資料集 - 合併補充資料集.csv",index=False)

In [ ]:
results=[]
for row in df_merge.iterrows():
    common_name_c = str(row[1]['common_name_c'])
    name_code = str(row[1]['name_code'])
    print("common_name_c=%s,name_code=%s" %(common_name_c,name_code))
    [taxonUUID,total]=getcnt_by_namecode(name_code)
    result=[common_name_c,name_code,taxonUUID,total]
    print(result)
    results.append(result)

In [ ]:
for result in results:
    print("%s,%s,%s,%s" %(result[0],result[1],result[2],result[3]))

# 多選展開

In [ ]:
# 多選展開
# ex: 都市區, 農業區
col_sel=['棲地類型Habitat types','類別Life form types']

def multisel_expand(df,col_sel):
    results = []
    for col_name in col_sel:
        df_col=df[col_name]
        cells = df_col[df_col.notna()].tolist()
        for cell in cells:
            values = cell.split(",")
            values = list(map(str.strip,values))
            for v in values:
                result = [col_name,v]
                results.append(result)
    return results

results = multisel_expand(df_extend,col_sel)

for r in results:
    print("%s,%s" %(r[0],r[1]))

# 多選項目編碼

In [19]:
items_str="""1-病毒
2-細菌
3-古箘
4-真菌
5-藻類
6-原生生物
"""

items = items_str.split("\n")
i=1

if items_str.find("其他")>=0:
    print("99-其他")
    
for item in items:
    if item.strip()!="":
        if len(item)>0:
            cols=item.split("-")
            if len(cols)==2:
                if not cols[1]=='其他':
                    print("%i-%s" %(i,cols[1].strip()))
                    i+=1
            else:
                if not cols[0]=='其他':
                    print("%i-%s" %(i,item.strip()))
                    i+=1

1-病毒
2-細菌
3-古箘
4-真菌
5-藻類
6-原生生物


In [ ]:
txt="""BaseElevation(基準高)、County(縣市名稱)、CountyIdentifier(縣市代碼)、DisuseDate(廢站日期)、EstablishDate(設站日期)、FinishDate(完鑿日期)、FinishDepth(完鑿深度)、GroundHeight(距地面高)、GroundwaterLayerCode(地下水分層代碼)、GroundwaterZone(地下水分區名稱)、GroundwaterZoneIdentifier(地下水分區代碼)、Identifier(井號)、LayerAttribute(含水層屬性，C為受壓 L為漏壓 U為自由)、LocationAddress(站址)、LocationByTWD67(TM2度分帶67坐標)、LocationByTWD97(TM2度分帶97座標，TM2度分帶97坐標)、ModificationDate(異動日期)、ModificationReason(異動別，1為設站 2為遷站 3為撤站 4為站況異動 5為其他異動 6為復站)、OrganizationIdentifier(管理單位代碼)、PipeMaterial(井管材質，1為不鏽鋼管 2為PVC4 3為鍍鋅鋼管 4為鋼管 5為鐵管 6為塑膠管 7為黑鐵管)、Remarks(備註)、StartDateOfWaterLevelObservation(水位開始觀測日期)、Town(鄉鎮名稱)、TownIdentifier(鄉鎮代碼)、WaterLevel(水位高)、WaterResourcesDistrictName(水資源分區名稱)、WellDepth(井深)、WellElevation(目前井頂高)、WellName(井名稱)、WellNumber(孔數)、WellRadius(井徑)、WellType(井別)"""
cols=txt.split('、')
for col in cols:
    print(col.strip().replace(" ",""))

# Generate Report

In [2]:

def rpt_gen(df_rows,filename_prefix="rpt"):

    import datetime
    datetime_dt = datetime.datetime.today()# 獲得當地時間
    datetime_str = datetime_dt.strftime("%Y/%m/%d %H:%M:%S")  # 格式化日期
    for row in df_rows.iterrows():
        lines_m=[]
        lines=[]        
        name_code = str(row[1]['name_code'])
        common_name_c = str(row[1]['common_name_cL'])
        filename_name = "%s-%s" %(name_code,common_name_c)
        lines.append("# %s (%s)" % (name_code,common_name_c))
        lines.append("\nreport generated on %s by %s %s\n" %(datetime_str,TITLE,VERSION))
        row1 = row[1]
        lines.append("| 欄位名稱 | 欄位中文 | 欄位英文 | 欄位值 |")
        lines.append("| -------------|-------------|-------------|-------------|")
        for i in range(len(row1.index)):
            value_str=str(row1.values[i])
            col_name = row1.index[i]
            idx = df_coldef['name']== col_name
            if value_str.find("\n")>0:
                if True in idx.value_counts():
                    eng = df_coldef[idx]['eng'].values.tolist()[0]
                    ch = df_coldef[idx]['ch'].values.tolist()[0]
                    lines_m.append("## %s | %s | %s" %(col_name,ch,eng))
                else:
                    lines_m.append("## %s" %(col_name))
                lines_m.append("%s" %(value_str.replace("\n","\n\n")))
            
            else:
                
                
                
                
                if value_str=="nan":
                    value_str=""
                
                
                
                if True in idx.value_counts():
                    #print('exist')
                    eng = df_coldef[idx]['eng'].values.tolist()[0]
                    ch = df_coldef[idx]['ch'].values.tolist()[0]
                    lines.append("| %s | %s | %s | %s |" %(col_name,ch,eng,value_str))
                else:
                    eng =''
                    ch=''
                    lines.append("| %s | | | %s |" %(col_name,value_str))
                
        pathname='output/%s_%s.md' %(filename_prefix,filename_name)
        with open(pathname, 'w') as f:
            f.write('\n'.join(lines))
            f.write('\n')
            f.write('\n')
            f.write('\n'.join(lines_m))
            
        print("%s saved" %(pathname))

if 0:
    idx=df_merge.index[df_merge['name_code']=='403968']
    df_rows = df_merge.loc[idx]
    rpt_gen(df_rows,"rpt")
if 1:
    idx=df_merge.index[(df_merge['common_name_cR'].notna()) & (df_merge['name_code']!='nan')]
    df_rows = df_merge.loc[idx]
    rpt_gen(df_rows,"rpt")

output/rpt_340020-蘇鐵白輪盾介殼蟲.md saved
output/rpt_426961-松材線蟲.md saved
output/rpt_403968-中國梨木蝨.md saved
output/rpt_422619-多線真稜蜥;多線南蜥;多紋南蜥.md saved
output/rpt_313411-河殼菜蛤.md saved
output/rpt_315085-福壽螺.md saved
output/rpt_416300-緬甸小鼠;波里尼西亞鼠.md saved
output/rpt_401070-紅入侵火家蟻;入侵紅火蟻.md saved
output/rpt_417599-小花蔓澤蘭.md saved
output/rpt_Kew-259467-布袋蓮;浮水蓮花;鳳眼蓮.md saved


In [36]:
idx=df_merge.index[(df_merge['common_name_cR'].notna()) & (df_merge['name_code']!='nan')]
df_rows = df_merge.loc[idx]
df_rows

,name_code,kingdom,kingdom_c,phylum,phylum_c,class,class_c,order,order_c,family,...,introduced_range,habitat_description,nutrition,reproduction,life_cycle,species_status,suggestion,note,reference,images
24.0,340020,Animalia,動物界,Arthropoda,節肢動物門,Insecta,昆蟲綱,Hemiptera,半翅目,Diaspididae,...,台灣、香港和新加坡、美國的佛羅里達州和夏威夷群島，以及中美洲加勒比海區域的開曼群島、美屬維京...,主要寄生在蘇鐵科(Cycadaceae)、美洲鳳尾蕉科(Zamiaceae)和蕨狀蘇鐵科(S...,植食性。食蘇鐵科 (Cycadaceae)與蕨蘇鐵科 (Stangeriaceae)的多種植...,雌成蟲於第36天開始達性成熟可開始產卵。雌成蟲可終生產卵，並在繁殖季節，產卵期約4-6週，而...,蘇鐵白輪盾介殼蟲從卵到成蟲大約30-40天。卵長徑約0.23 mm，橫徑約0.11mm。在定...,NaN,台灣全島皆有分佈。\nDistributed in all of Taiwan,此種介殼蟲為直接變態，若蟲及成蟲均以刺吸式口器吸食植物的枝條及葉片。若蟲於母蟲的介殼下孵化，...,1. 邱一中、吳文哲、賴博永、石正人。2004。蘇鐵白輪盾介殼蟲之發生生態及防治。台灣作物...,NaN
39.0,403968,Animalia,動物界,Arthropoda,節肢動物門,Insecta,昆蟲綱,Hemiptera,半翅目,Psyllidae,...,台灣,成蟲與若蟲均刺吸梨樹的芽、葉及嫩稍。除直接消耗營養外，受害葉出現褐色枯斑，甚至全葉變黑，嚴重...,植食性昆蟲，刺吸式口器取食，以梨樹為主。\nPhytophagous insect. Bot...,卵生\nOviparous.,一年發生多代且世代重疊，以成蟲在樹枝的裂縫、切口、雜草、落葉或土隙中越冬。越冬成蟲在2月間開...,在台灣全年可見。危害分佈中部和平、東勢、和平、新社、卓蘭及仁愛等地區的梨園。具有五大特性:不...,NaN,目前因應策略仍以藥劑防治為主。即以9.6%亦達胺溶液3000倍或20%亞滅培可溶性粉劑400...,1. 王文哲。2005。中國梨木蝨之生態與防治。梨栽培管理技術研討會專輯 P355-366。...,NaN
75.0,422619,Animalia,動物界,Chordata,脊索動物門,Reptilia,爬蟲綱,Squamata,有鱗目,Scincidae,...,台灣 (1992) 及美國佛羅里達州 (1990)。[5],常出現在台灣南部的檳榔園、椰子園、香蕉園、果園、廢耕地、公園、灌溉溝渠旁、道路旁草叢、河床、...,雜食性，以昆蟲為主要食物來源，特別是鞘翅目、直翅目及雙翅目，此外亦捕食蛛型綱的蜘蛛、唇足綱的...,胎生。[2] 每胎平均可產下6隻仔蜥(4-12隻)。[1] 在中國海南島，繁殖中的雌蜥的最小...,壽命約3-5年，沒有冬眠或休眠現象，全年活動，一年可生殖2次，每年5月及11月為生殖高峰期，...,廣泛分布於台灣西南部，從雲林濁水溪以南至恆春龍鑾潭以北，海拔500m以下的區域幾乎都可見其蹤...,NaN,NaN,1.\t朱賢斌。2000。入侵南台灣的異族-多線南蜥。自然保育季刊 29: 50-53。\n...,NaN
96.0,313411,Animalia,動物界,Mollusca,軟體動物門,Bivalvia,雙殼綱,Mytiloida,貽貝目,Mytilidae,...,日本、美洲、台灣、阿根廷、巴西、巴拉圭、烏拉圭 [7],棲息於河流及河口半淡鹹水區域。能適應的鹽度相當廣，除淡水之外，尚可忍受河口或沿海地區鹽度千分...,濾食性。攝食方法通常以鰓過濾浮游性微細植物 (如藻類) 和有機顆粒，再由鰓上的食物溝送入口中...,於16-28°C (六月到九月) 產卵，一年約產卵1-2次。雌雄異體，將配子排入水中並進行體...,受精卵發育為第一期幼蟲叫「擔輪幼蟲」，再發育為第二期幼蟲叫「面盤幼蟲」，開始攝食浮游生物，經...,在台灣分佈於日月潭、石門水庫、直潭。石門水庫有發現較正常體型兩倍大 (約5、6公分) 的河殼...,NaN,河殼菜蛤可容忍的鹽度廣，因此在沿海地區活動的船舶成為潛在的播遷工具，尤其是大型的輪船需要大量...,1.\t吳錫圭、蔡奇立、林旭宏。2003。入侵台灣的河殼菜蛤。自然保育季刊 41: 32-3...,NaN
146.0,315085,Animalia,動物界,Mollusca,軟體動物門,Gastropoda,腹足綱,Mesogastropoda,中腹足目,Ampullariidae,...,美國、中國、東南亞、日本、台灣,棲息於南北半球、熱帶及亞熱帶地區之淡水或半淡鹹水及湖沼、池塘、溝渠等靜水水域。棲息於10-1...,雜食性，但偏向植食。攝食種類甚廣，包括水生植物、陸生蔬菜類，甚魚類屍體[4]。福壽螺偏好的綠...,雌雄異體、卵生。雌螺在夜間或黎明於水面上乾燥物體的表面，如稻桿、溝壁、牆壁、田埂、雜草等上生...,成螺壽命可達2-6年。一年可繁殖2-4個世代，完成一世代需時88-204天，隨季節及食物而異...,分布於台灣全島中、低海拔水域[1]。,NaN,NaN,1.\tGlobal invasive species database (GISD). P...,NaN
159.0,416300,Animalia,動物界,Chordata,脊索動物門,Mammalia,哺乳綱,Rodentia,嚙齒目,Muridae,...,一支向西散佈到東南亞的越南、寮國、泰國、緬甸等地，另一支則隨毛利人與波里尼西亞人播遷至大洋洲...,喜棲息於地被植物繁密與排水良好的地方，但也會出現在旱作、廢耕地、林地，甚至環礁島上。在草生地...,雜食性，以植物為主，另外會捕食小型脊椎動物、包括蜥蜴、鳥類[2]。攝食範圍廣，包括各種闊葉植...,"胎生。雌鼠可多次發情與自發性排卵[7, 9]。繁殖行為大部分取決於食物的充足與否。緬甸小鼠大...",生殖具季節性，每年春夏季或夏季到冬初為繁殖季。幼鼠於28天左右斷奶 [9]。,目前緬甸小鼠在花蓮地區的分布範圍不大，在受到天然與人為地理屏障的限制下，向外擴展的程度有限（...,NaN,緬甸小鼠的活動力低，少有長距離移動與轉換活動範圍的情形 [3],1. 自然與人文數位博物館。2013。緬甸小鼠。(http://digimuse.nmn...,NaN
166.0,401070,Animalia,動物界,Arthropoda,節肢動物門,Insecta,昆蟲綱,Hymenoptera,膜翅目,Formicidae,...,被引進澳洲與北美的部分地區。也被引進到一些脆弱的島嶼生態系，包括一些加勒比海島嶼 (波多黎各...,農業環境如水稻田、蔬菜園、園藝場、花卉植栽栽培區、休耕田、農舍、竹林、養雞廠。及都市環境如公...,雜食性，除對土棲的動物造成捕食傷害外，在發生嚴重的地區，甚至土壤中的蚯蚓都被捕食殆盡，造成生...,入侵紅火蟻並沒有特定的婚飛時期 (交配期)，只要蟻巢成熟全年都可以有新的生殖個體形成。雌雄蟻...,成熟蟻巢中的蟻后每天約產1500-5000個卵，卵發育至成蟲約需20-45天 (工蟻)、30...,在新北市林口區、桃園市六鄉鎮（桃園、蘆竹、大溪、龜山、八德、大園）、嘉義縣水上鄉有發現案例[...,NaN,NaN,1. 國家紅火蟻防治中心網頁 (https://fireant.baphiq.gov.tw/...,NaN
224.0,417599,Plantae,植物界,Tracheophyta,維管束植物門,Magnoliopsida,木蘭綱,Asterales,菊目,Asteraceae,...,1950 年以後，被引入南亞及東南亞為地被植物；幾十年來已由引種地區向外擴散，在南亞、東南亞...,小花蔓澤蘭在熱帶美洲常見於受干擾的環境、潮濕的土地或沼澤地區；在南美洲則分布於潮濕的森林和淡...,NaN,種子繁殖；無性繁殖能力亦強，蔓莖接觸土壤的每個節可長出根及新芽、節間亦能長出不定根。[8],多年生藤蔓。夏季為小花蔓澤蘭之生長旺盛其，植株迅速擴散，冬季呈乾枯狀，為種子散播期；各地花果...,小花蔓澤蘭在全台17 個縣市、130 個鄉鎮的農地皆有紀錄，危害範圍逐年增加 [6]。常入侵...,NaN,小花蔓澤蘭所能發芽的溫度範圍比原生的蔓澤蘭廣，種子發芽率及生長勢顯著高於蔓澤蘭，而營養生長期...,[1]\t郭耀綸、陳志遠、林杰昌。2002。藉連續切蔓法及相剋作用防治外來入侵的小花蔓澤蘭。...,NaN
232.0,Kew-259467,Plantae,植物界,Tracheophyta,維管束植物門,Magnoliopsida,木蘭綱,Commelinales,鴨跖草目,Pontederiaceae,...,已擴散到全球熱帶及亞熱帶水域 [16],布袋蓮對水質條件的需求很低，只要有水、光照、營養鹽，就能夠生長 [3]。常見於高日照、溫暖的...,NaN,布袋蓮是自由飄浮的水生植物，可以透過營養繁殖，透過匍匐莖上腋芽分裂植株或是藉由種子繁殖[20...,布袋蓮為多年生植物。種子沉于水底後休眠期長達15 年。種子發芽後，初期之幼苗藉根固著在泥土

In [53]:
#idx = df_coldef['name']== 'kingdom'
#df_coldef.columns 
# in df_coldef['name']
#df_coldef['name'].str.find('animal')
#df_coldef['name']
#index = pd.Index([3, 1, 2, 3, 4])
#index.value_counts()
idx = df_coldef['name']== 'life_type'
True in idx.value_counts()

True

In [12]:
import datetime
datetime_dt = datetime.datetime.today()# 獲得當地時間
datetime_str = datetime_dt.strftime("%Y/%m/%d %H:%M:%S")  # 格式化日期

In [14]:
TITLE

'TiasdSWAK'

In [7]:
idx = df_coldef['name']=='life_type'
df_coldef[idx]['eng'].values.tolist()[0]

'Life form types'